In [1]:
import os
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import examples.utils as utils

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [26]:
learning_rate = 0.01
batch_size = 128
epochs = 30
n_train = 60000
n_test = 10000

In [28]:
train, val,test = utils.read_mnist('data/mnist', flatten = True)

In [29]:
tf.reset_default_graph()

In [35]:
print(type(train))
print(np.shape(train[0]))
print(np.shape(train[1]))

<class 'tuple'>
(55000, 784)
(55000, 10)


In [36]:
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(100)
test_data = tf.data.Dataset.from_tensor_slices(test)

In [38]:
train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [39]:
iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
img, label = iterator.get_next()

In [40]:
train_data.output_types

(tf.float32, tf.float64)

In [41]:
train_data.output_shapes

(TensorShape([Dimension(None), Dimension(784)]),
 TensorShape([Dimension(None), Dimension(10)]))

In [44]:
train_init =iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

In [42]:
w = tf.get_variable(shape = (784, 10), name = 'weights', initializer = tf.random_normal_initializer(0,0.01))
b = tf.get_variable(shape = (1, 10), name = "bias", initializer = tf.zeros_initializer())

In [47]:
logits  = tf.matmul(img,w ) + b

In [48]:
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = logits)
loss = tf.reduce_mean(entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [49]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [50]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label,1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [ ]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    
    for i in range(epochs):
        sess.run(train_init)
        total_loss = 0
        n_batches = 0
        
        try :
            while True:
                _,l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        
        print("avg loss for epoch {0} is {1}".format(i,total_loss/n_samples))
    
    print("Time", time.time() - start_time) 
    
    
    
    sess.run(test_init)
    total_correct_preds = 0
    
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
    
    
    
    print("Acc")
    